<a href="https://colab.research.google.com/github/PawanRoy712/pa/blob/main/youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as ms
%matplotlib inline

In [10]:
train_data = []

In [11]:
data_files = ['/content/Youtube01-Psy.csv']
for file in data_files:
    data = pd.read_csv(file)
    train_data.append(data)
train_data = pd.concat(train_data)

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  350 non-null    object
 1   AUTHOR      350 non-null    object
 2   DATE        350 non-null    object
 3   CONTENT     350 non-null    object
 4   CLASS       350 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 13.8+ KB


In [14]:
train_data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [15]:
train_data['CLASS'].value_counts()

1    175
0    175
Name: CLASS, dtype: int64

In [16]:
def drop_fectures(features,data):
    data.drop(features,axis=1,inplace=True)
drop_fectures(['COMMENT_ID','AUTHOR','DATE'],train_data)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CONTENT  350 non-null    object
 1   CLASS    350 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.6+ KB


In [18]:
import re
re.findall("[A-Za-z]+","Hey Pawan,How are you doing?")

['Hey', 'Pawan', 'How', 'are', 'you', 'doing']

In [19]:
def process_content(content):
    return " ".join(re.findall("[A-Za-z]+",content.lower()))

In [20]:
train_data['processed_content'] = train_data['CONTENT'].apply(process_content)
train_data.head()

,CONTENT,CLASS,processed_content
0,"Huh, anyway check out this you[tube] channel: ...",1,huh anyway check out this you tube channel kob...
1,Hey guys check out my new channel and our firs...,1,hey guys check out my new channel and our firs...
2,just for test I have to say murdev.com,1,just for test i have to say murdev com
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,me shaking my sexy ass on my channel enjoy
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1,watch v vtarggvgtwq check this out


In [21]:
drop_fectures(['CONTENT'],train_data)

In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data['processed_content'],train_data['CLASS'],test_size=0.2,random_state=57)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
x_train_counts = count_vect.fit_transform(x_train)
x_train_counts.shape

(280, 1028)

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
tranformer = TfidfTransformer()
x_train_tfidf = tranformer.fit_transform(x_train_counts)
x_train_tfidf.shape

(280, 1028)

In [25]:
x_test_counts = count_vect.transform(x_test)
x_test_counts

<70x1028 sparse matrix of type '<class 'numpy.int64'>'
	with 287 stored elements in Compressed Sparse Row format>

In [26]:
x_test_tfidf = tranformer.transform(x_test_counts)
x_test_tfidf

<70x1028 sparse matrix of type '<class 'numpy.float64'>'
	with 287 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train_tfidf,y_train)

LogisticRegression()

In [28]:
predictions = model.predict(x_test_tfidf)

In [29]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
confusion_matrix(y_test,predictions)

array([[41,  2],
       [ 2, 25]])

In [30]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        43
           1       0.93      0.93      0.93        27

    accuracy                           0.94        70
   macro avg       0.94      0.94      0.94        70
weighted avg       0.94      0.94      0.94        70



In [31]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train_tfidf,y_train)

RandomForestClassifier()

In [32]:
predictions = model.predict(x_test_tfidf)

In [33]:
confusion_matrix(y_test,predictions)

array([[42,  1],
       [ 3, 24]])

In [34]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95        43
           1       0.96      0.89      0.92        27

    accuracy                           0.94        70
   macro avg       0.95      0.93      0.94        70
weighted avg       0.94      0.94      0.94        70



In [38]:
from sklearn.model_selection import GridSearchCV
parameters = {
                     'max_depth' : [1,2],
                     'n_estimators': [10,20],
                     'max_features': ['sqrt', 'auto', 'log2'],
                     'min_samples_split': [10,20],
                     'min_samples_leaf': [1, 3],
                     'bootstrap': [True, False],
                     }
model = GridSearchCV(RandomForestClassifier(),parameters)
model.fit(x_train_tfidf,y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForest

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False], 'max_depth': [1, 2],
                         'max_features': ['sqrt', 'auto', 'log2'],
                         'min_samples_leaf': [1, 3],
                         'min_samples_split': [10, 20],
                         'n_estimators': [10, 20]})

In [39]:
model.best_params_

{'bootstrap': False,
 'max_depth': 2,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 20}

In [40]:
predictions  = model.predict(x_test_tfidf)

In [41]:
confusion_matrix(y_test,predictions)

array([[12, 31],
       [ 0, 27]])

In [42]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      0.28      0.44        43
           1       0.47      1.00      0.64        27

    accuracy                           0.56        70
   macro avg       0.73      0.64      0.54        70
weighted avg       0.79      0.56      0.51        70

